# transfer.py

In [ ]:
%%writefile transfer.py

import sys
import requests

def post_form(url, amount, password):
    r = requests.post(url, data = {'transfer':amount,'password':password})
    print(r.text)

if __name__ == "__main__":
    #sys.argv = ["transfer.py", "localhost", "1,000$","1234"]
    if len(sys.argv) != 4:
        print( 'Missed argument.\n usage:`python transfer.py <ip_address> <amount> <password>`')
        sys.exit(-1)
    url = 'http://' + sys.argv[1] + '/'
    post_form(url, sys.argv[2], sys.argv[3])

# web-server.py

In [ ]:
%%writefile web-server.py

from BaseHTTPServer import BaseHTTPRequestHandler, HTTPServer
import SocketServer

get_res = "<html><body><h1>Devil Hi!</h1></body></html>"

class S(BaseHTTPRequestHandler):
    password = None
        
    def _set_headers(self):
        self.send_response(200)
        self.send_header('Content-type', 'text/html')
        self.end_headers()
        
    def do_GET(self):
        self._set_headers()
        #self.wfile.write("<html><body><h1>Devil Hi!</h1></body></html>")
        self.wfile.write(get_res)

    def do_HEAD(self):
        self._set_headers()
        
    def extract_pass(self,data):
        return data[data.find("password=")+1:]
    
    def extract_money(self,data):
        return data[data.find('=')+1:data.find('&')].replace("%2C",",").replace("%24","$")
    
    def do_POST(self):
        content_length = int(self.headers['Content-Length']) # <--- Gets the size of data
        post_data = self.rfile.read(content_length) # <--- Gets the data itself
        print("POST request,\nPath: %s\nHeaders:\n%s\n\nBody:\n%s\n",
            str(self.path), str(self.headers), post_data.decode('utf-8'))
        print(post_data)
        self._set_headers()
        
        if "transfer" in post_data:
            if S.password is None:
                self.wfile.write("Error: You should set password".encode('utf-8'))
            elif S.password == self.extract_pass(post_data):
                self.wfile.write("Transfered {} successfully.".format(
                    self.extract_money(post_data)).encode('utf-8'))
            else :
                self.wfile.write("Error: Password is incorrect.".encode('utf-8'))
        elif "pass" in post_data:
            S.password = self.extract_pass(post_data)
            self.wfile.write("Password was set.".encode('utf-8'))
        else:
            self.wfile.write("Error: Unknown key.".encode('utf-8'))

def run(server_class=HTTPServer, handler_class=S, port=80):
    server_address = ('', port)
    httpd = server_class(server_address, handler_class)
    print 'Starting httpd...'
    httpd.serve_forever()
        
if __name__ == "__main__":
    run()

# set_password.py

In [ ]:
%%writefile set_password.py

import sys
import requests

def post_form(url, value):
    r = requests.post(url, data = {'password':value})
    print(r.text)

if __name__ == "__main__":
    #sys.argv = ["transfer.py", "localhost", "1234"]
    if len(sys.argv) != 3:
        print( 'Missed argument.\n usage:`python set_password.py <ip_address> <password>`')
        sys.exit(-1)
    url = 'http://' + sys.argv[1] + '/'
    int(sys.argv[2])
    post_form(url, sys.argv[2])

In [15]:
%%writefile test.py

from scapy.all import *
from netfilterqueue import NetfilterQueue
import codecs


similar="" #Search for common bytes in the packets
newData = "" #Insert new Data

def modify(packet):
    pkt = packet.get_payload()
    pktInfo = IP(pkt)
    pktHex = pkt.hex()

    #Filter out the packets if they are TCP Protocol and going to destination port 7777. 
    #The iptable command should already be filtering by destination port 7777
    if pktInfo.haslayer(TCP) and pktInfo.getlayer(TCP).dport == 7777 and similar in pktHex:
        pktDefault = pktHex[:104]
        pktEdit = pktHex[104:]

        pktSave = pktEdit[16:]
        pktSave = newData + pktSave
        pktNew = pktDefault + pktSave

        pktBack = bytes.fromhex(pktNew) # Convert back from hex
        pktBack = IP(pktBack) 
        del pktBack[IP].chksum   #After deleting scapy will automatically recalculate the IP and TCP Checksum
        del pktBack[TCP].chksum
        packet.set_payload(bytes(pktBack)) #Set the payload and send the packet
        print("Payload sent!")
    packet.accept()

nfqueue = NetfilterQueue()
nfqueue.bind(0, modify)  #Bind to Queue 0 and intercept traffic passing through queue 0
try:
    print ("[*] waiting for data")
    nfqueue.run()
except KeyboardInterrupt:
    pass

nfqueue.unbind()

Writing test.py
